In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 3.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ P

In [2]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')
import re

Mounted at /content/drive


# 임의의 리스트

In [10]:
song_id_list = """
30581594
32720013
2318381
2318380
30620447
33315851
1756326
1373945
1373941
1373946
1373948
1373964
4033699
4033700
4033701
4033702
4033703
4033708
4033706
4033709
4033711
4033712
4033713
4846881
4846884
4846889
3773425
5770943
1172483
2304249
30574632
30230511
11668926
978985
1110892
4420975
4420976
32594186
32252805
31196668
3803261
611100
111572
51052
69302
175431
175543
175571
4828185
4828186
4828187
4828188
4828190
30840261
528547
1576582
516679
10687
1763641
3836568
531748
3822
667278
"""
song_id_list = song_id_list.split('\n')[1:-1]

In [85]:
def getMylist(song_num, driver, col): 
  body = driver.find_element_by_css_selector('body')
  res = driver.page_source
  soup = bs(res, "html.parser")
  content = soup.find('div', {'id':'conts'})

  #앨범이름
  meta_div = content.find('div', {'class':'meta'})
  try:
    album_title_dl = meta_div.find('dl', {'class' : 'list'}).find('dd').find('a')
    album_title = album_title_dl.text
  except:
    print('al_title err:', song_num)
    album_title = ''

  #앨범번호
  try:
    al_href = album_title_dl['href']
    album_num = al_href.split("'")[-2]
  except:
    print('album_num err', song_num)
    album_num = ''

  #가수, 가수번호
 # try:
  info_div = content.find('div', {'class':'info'})
  artist_div = info_div.find('div', {'class' : 'artist'}).find_all('a')
  artist_set = set()
  artist_list = []
  artist_num_list = []
  for artist_item in artist_div:
    artist_tuple = (artist_item.text, artist_item['href'].split("'")[-2])
    artist_set.add(artist_tuple)
  
  for set_item in artist_set:
      artist_list.append(set_item[0])
      artist_num_list.append(set_item[1])
  artist = (' , ').join(artist_list)
  artist_num = (' ').join(artist_num_list)
 # except:
  #  print('artist err')
   # artist = ''
    #artist_num=''

  #노래 제목 찾기
  try:
    song_title = info_div.find('div', {'class' : 'song_name'}).text
    song_title = song_title.replace('\t', '')
    song_title = song_title.split('\n')[-2]
  except:
    print('song_title err', song_num)
    song_title = ''

  #장르 찾기
  meta_div = content.find('div', {'class':'meta'})
  genre_dt = meta_div.find('dl', {'class' : 'list'}).text.split('\n')
  genre = genre_dt[genre_dt.index('장르')+1]

  #좋아요 수
  try:
    d_song_like = content.find('div', {'class':'d_song_like'}).find('button').find_all('span')[1]
    luv = d_song_like.text

  except:
    print('luv err', song_num)
    luv=''
  try:
    #노래에 추천 태그 긁어오기
    div_service_list_album = soup.find('div', {'class':'service_list_album'})
    tag_ul = div_service_list_album.find('ul')
    tag_div = tag_ul.find_all('div', {'class':'tag_list'})
    tag_list= []

    #긁어온 태그 리스트화
    for div in tag_div:
      tag_list += div.text.replace('\n', ' ')[1:-1].split(' ')


    #새로운태그 set화하여 중복 제거 및 공백태그제거

    tag_set = set(tag_list)
    if '' in tag_set:
      tag_set.remove('')
    tag = (' ').join(tag_set)
  except:
    tag = ''
    print('tag err', song_num)

            #('번호', '앨범번호', '앨범이름', '가수리스트(띄어쓰기구분)', '가수번호', '노래제목', '장르', '좋아요', '태그')
  return pd.DataFrame([song_num, album_num,  album_title, artist, artist_num, song_title, genre, luv, tag], index=col).transpose()

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

main_url = 'https://www.melon.com/song/detail.htm?songId='


col = ['번호', '앨범번호', '앨범이름', '가수', '가수번호', '노래제목', '장르', '좋아요', '태그']
df = pd.DataFrame(columns=col)

for song in song_id_list:
  url = main_url + song
  driver.get(url)
  time.sleep(3)

  df = pd.concat([df, getMylist(song, driver, col)], axis=0)
  print(song_id_list.index(song)+1, '/', len(song_id_list))
  print(df.iloc[song_id_list.index(song)])
df
df.to_csv('/content/drive/MyDrive/Colab Notebooks/세미프로젝트/data/Mylist.csv')

#테스트

In [86]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

main_url = 'https://www.melon.com/song/detail.htm?songId='


col = ['번호', '앨범번호', '앨범이름', '가수', '가수번호', '노래제목', '장르', '좋아요', '태그']
df = pd.DataFrame(columns=col)


song = '1373948'
url = main_url + song
driver.get(url)
time.sleep(3)

df = pd.concat([df, getMylist(song, driver, col)], axis=0)
print(song_id_list.index(song)+1, '/', len(song_id_list))
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


tag err 1373948
11 / 63


,번호,앨범번호,앨범이름,가수,가수번호,노래제목,장르,좋아요,태그
0,1373948,325744,Good Night Vol.2,Sade,28744,Kiss Of Life,POP,4,
